Try different learning rates for GD vs MRGD

* Use cosine similarity as metric


In [1]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9
max_shift = 64
lmbda = 1e-30
#noise = 1e30
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
#image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
#I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
I_out = 10*np.array([0, 0, 0, 10, 5,5,300])
#I_out = 10*np.array([0, 0, 0, 0, 0,0,320])



/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LR = 1e-1
model = MultiResSolver(multires, 
                        loss, 
                        LR = LR,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_)
model.solve_multigrid()
model.print_time()

----------- s = 9 -----------
----------- s = 8 -----------
----------- s = 7 -----------
----------- s = 6 -----------
1
Iter 1, [loss,mse,reg,csim] : [129134.6449206, 129134.6449206, 0.0, 0.0026169, ] 
LR = 0.1
1
Iter 2, [loss,mse,reg,csim] : [1725590.9143184, 1725590.9143184, 0.0, 0.0461617, ] 
LR = 0.1
1
Iter 3, [loss,mse,reg,csim] : [40946562.116732, 40946562.116732, 0.0, 0.029473, ] 
LR = 0.1
1
Iter 4, [loss,mse,reg,csim] : [2130276058.2800107, 2130276058.2800107, 0.0, 0.0283141, ] 
LR = 0.1
1
Iter 5, [loss,mse,reg,csim] : [137113529826.70746, 137113529826.70746, 0.0, 0.0282135, ] 
LR = 0.1
1
Iter 6, [loss,mse,reg,csim] : [10055979235735.61, 10055979235735.61, 0.0, 0.0282959, ] 
LR = 0.1
1
Iter 7, [loss,mse,reg,csim] : [804838869041153.5, 804838869041153.5, 0.0, 0.0283924, ] 
LR = 0.1
1
Iter 8, [loss,mse,reg,csim] : [6.857479515156807e+16, 6.857479515156807e+16, 0.0, 0.0284858, ] 
LR = 0.1
1
Iter 9, [loss,mse,reg,csim] : [6.124343595898485e+18, 6.124343595898485e+18, 0.0, 0.02857

KeyboardInterrupt: 

In [ ]:
np.save("experiments/results/exp3/time_gd.npy",np.array(model.iter_times))

In [ ]:
mrgd_time = np.load("experiments/results/exp3/time_mrgd.npy")
gd_time = np.load("experiments/results/exp3/time_gd.npy")

In [ ]:
mrgd_time = np.cumsum(mrgd_time)
gd_time = np.cumsum(gd_time)

In [ ]:
plt.plot(mrgd_time, label = "MRGD")
plt.plot(gd_time, label = "GD")

In [ ]:
mrgd_loss = np.load("experiments/results/exp3/cpwc_overlap50_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
gd_loss = np.load("experiments/results/exp3/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.1_loss.npy")

In [ ]:
plt.semilogy(mrgd_loss, label = "MRGD")
plt.semilogy(gd_loss, label = "GD")

In [ ]:
plt.figure(figsize=(10, 5))
plt.semilogy(mrgd_time, mrgd_loss[:-7], label='mrgd')
plt.semilogy(gd_time, gd_loss[:-7], label='gd')
plt.xlabel('Time (s)')
plt.ylabel('Loss')
plt.title('Loss vs Time')
plt.legend()
plt.grid()

In [ ]:
if max_shift == 32:
    overlap = 75
elif max_shift == 64:
    overlap = 50
else:
    overlap = 25
from src.utils.manage_data import save_data,unwrap_2d,extract_data

path = save_data(
          path = "experiments/results/exp3",
          model = model,
          image_path= "samples/potential.npy",
          metrics = ['csim','loss','frc',"image"],
          device = "cuda",
          max_scale = 9,
          overlap = overlap,
          spline_type="cpwc",
          lambda_ = lmbda,
          lr = LR,
          noise_type="poisson",
          noise = "None",
          loop = "mrgdp")



In [ ]:
mrgd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_csim.npy")
mrgdp01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_csim.npy")
mrgd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_csim.npy")
gd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_csim.npy")
gd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_csim.npy")
gd1 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.1_csim.npy")

In [ ]:
plt.figure(figsize=(15,7))
plt.title("1-Cosine Similarity")
plt.semilogy(1-mrgd001,label="MRGD w/ LR= 0.001")
plt.semilogy(1-mrgd01,label="MRGD w/ LR= 0.01")
plt.semilogy(1-mrgdp01,label="MRGD w/ LR= 0.01 x 2^(scale - min_scale)")
plt.semilogy(1-gd001,label="GD w/ LR= 0.001")
plt.semilogy(1-gd01,label="GD w/ LR= 0.01")
plt.semilogy(1-gd1,label="GD w/ LR= 0.1")
plt.xlabel("Iterations")
plt.ylabel("Cosine Similarity")
plt.grid()
plt.legend()
plt.show()


In [ ]:
mrgd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_loss.npy")
mrgd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
mrgdp01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
gd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_loss.npy")
gd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
gd1 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.1_loss.npy")

In [ ]:
plt.figure(figsize=(10,5))
plt.title("MSE")
plt.semilogy(mrgd001,label="MRGD w/ LR= 0.001")
plt.semilogy(mrgd01,label="MRGD w/ LR= 0.01")
plt.semilogy(mrgdp01,label="MRGD w/ LR= 0.01 x 2^(scale - min_scale)")
plt.semilogy(gd001,label="GD w/ LR= 0.001")
plt.semilogy(gd01,label="GD w/ LR= 0.01")
plt.semilogy(gd1,label="GD w/ LR= 0.1")
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.grid()
plt.legend()
plt.show()
